# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df= pd.read_csv('homepage_actions.csv')

In [2]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [6]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [7]:
clickers = set(df[df.action=='click']['id'].unique())
viewers = set(df[df.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber of clickers: {}".format(len(viewers), len(clickers)))
print("Number of Viewers who didn't click: {}".format(len(viewers-clickers)))
print("Number of Clickers who didn't view: {}".format(len(clickers-viewers)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


Everyone who clicked viewed. 4468 viewers didn't click

In [8]:
experiment = set(df[df.group=='experiment']['id'].unique())
control = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(experiment&control)))

Overlap of experiment and control groups: 0


There is no overlap between the experiment group and control group

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [11]:
x=df.groupby('group')
x.describe()

id                                                     \
             count           mean            std       min        25%   
group                                                                   
control     4264.0  562817.716463  219435.742215  182994.0  377519.25   
experiment  3924.0  566744.854485  218714.595215  182988.0  371700.25   

                                          
                 50%       75%       max  
group                                     
control     561478.0  755831.5  937217.0  
experiment  574541.0  762061.0  937139.0

In [15]:
#Your code here
df['count'] = 1
df.head()
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


The average click rate for the control group is higher than that of the control group.

$H_0$:The average click rate for the control group is less than or equal to that of the control group.

$H_1$:The average click rate for the control group is greater than that of the control group.


Thus we'll do a one tailed test

In [19]:
print("variance click:\tControl: {}\tExperiment: {}".format(control.click.var(), experiment.click.var()))


variance click:	Control: 0.20153363056420093	Experiment: 0.2138749272820101


The two samples have different variances thus we use a welch test

In [22]:
from scipy import stats

In [27]:
test= stats.ttest_ind(control, experiment, equal_var=False)
test.pvalue


C:\Users\Admin\AppData\Local\Temp\ipykernel_14260\1215317143.py:1: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  test= stats.ttest_ind(control, experiment, equal_var=False)


array([0.00893281,        nan])

The pvalue is 0.0089 which is less than 0.05 thus we reject the null hypothesis concluding that we are 95% confident that The average click rate for the control group is greater than that of the control group.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [29]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks = control_rate * len(experiment)
print('expected_experiment_clicks:', expected_experiment_clicks)

expected_experiment_clicks: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [31]:
#Your code here
import numpy as np
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


The actual number of clicks was 24.57 standard deviations away from this expected number of clicks.

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [32]:
#computing the z-score
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks)/std
print(z_score)

3.6625360854823588


In [33]:
#Your code here
p_val = stats.norm.sf(z_score) 
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The pvalue is 0.001 which is also less than 0.05 thus rejecting the null hypothesis.This result matches that of the statistical test**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.